### Feature Generation

This notebook considers the scraped data and the projects of the x-ray images. It refines the dataset, as each report row now belongs to only a set of frontal and lateral images.
The visual feature extraction using DenseNet121 is done in this image.

In [58]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121

In [2]:
data = pd.read_csv('concatenated_data.csv')

In [3]:
projections = pd.read_csv('data_projections.csv')

In [5]:
data.head()

,image_id,image_caption,comparison,indication,findings,impression,image_count,indication_count,findings_count,impression_count
0,"CXR1_1_IM-0001-3001.png, CXR1_1_IM-0001-4001.png",ray chest pa and lateral,none,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x,2,3,33,3
1,"CXR10_IM-0002-1001.png, CXR10_IM-0002-2001.png",pa and lateral chest x,chest radiographs,male chest pain,the cardiomediastinal silhouette is within nor...,no acute cardiopulmonary process,2,3,38,4
2,"CXR100_IM-0002-1001.png, CXR100_IM-0002-2001.png",chest frontal lateral pm,none,no indication,both lungs are clear and expanded heart and me...,no active disease,2,2,10,3
3,"CXR1000_IM-0003-1001.png, CXR1000_IM-0003-2001...",pa and lateral chest x,pa and lateral chest radiographs,male,there is increased opacity within the right up...,increased opacity in the right upper lobe with...,3,1,52,36
4,"CXR1001_IM-0004-1001.png, CXR1001_IM-0004-1002...",chest frontal lateral pm,none,dyspnea subjective fevers arthritis immigrant ...,interstitial markings are diffusely prominent ...,diffuse fibrosis no visible focal acute disease,2,7,14,7
...,...,...,...,...,...,...,...,...,...,...
3909,"CXR995_IM-2478-1001.png, CXR995_IM-2478-1002.png",chest frontal lateral pm,NaN,sp cabg sob no rales,sternotomy sutures and bypass grafts have been...,post operative chest with no acute disease,2,5,23,7
3910,"CXR996_IM-2479-1001.png, CXR996_IM-2479-2001.png",pa and lateral chest with comparis,no comparison,chest pain,no findings,heart size is normal and lungs are clear no pn...,2,2,2,13
3911,"CXR997_IM-2479-1001.png, CXR997_IM-2479-2001.png",ray chest pa and lateral,chest x dated at hours,female transplant workup,calcified mediastinal no focal areas of consol...,no acute cardiopulmonary abnormality,2,3,23,4
3912,"CXR998_IM-2479-1001.png, CXR998_IM-2479-2001.png",pa and lateral chest x dated at am,none,with pain,cardiomediastinal silhouette demonstrates norm...,no acute cardiopulmonary abnormality,2,2,35,4


In [46]:
columns = ["front X-Ray", "lateral X-Ray", "findings"]

df_data = pd.DataFrame(columns = columns)

for data_row in tqdm(data.iterrows()):
    image_ids = data_row[1]['image_id'].split(',')
    
    # we will only consider the data if there are frontal and lateral projections
    frontal = []
    lateral = []
    for id in image_ids:
        imamge_name = id.strip()
        if projections[projections['Image_name'] == imamge_name]['label'].values[0] == "frontal":
            frontal.append(imamge_name)
        else:
            lateral.append(imamge_name)
    
    if len(frontal) > 0 and len(lateral) > 0:
        df1 = pd.DataFrame([[frontal[0], lateral[0], data_row[1]['findings']]], columns=columns)
        df_data = pd.concat([df_data, df1], ignore_index=True)


3914it [00:04, 857.15it/s]


In [48]:
df_data

,front X-Ray,lateral X-Ray,findings
0,CXR1_1_IM-0001-4001.png,CXR1_1_IM-0001-3001.png,the cardiac silhouette and mediastinum size ar...
1,CXR10_IM-0002-2001.png,CXR10_IM-0002-1001.png,the cardiomediastinal silhouette is within nor...
2,CXR100_IM-0002-1001.png,CXR100_IM-0002-2001.png,both lungs are clear and expanded heart and me...
3,CXR1000_IM-0003-1001.png,CXR1000_IM-0003-2001.png,there is increased opacity within the right up...
4,CXR1001_IM-0004-1001.png,CXR1001_IM-0004-1002.png,interstitial markings are diffusely prominent ...
...,...,...,...
3297,CXR995_IM-2478-1001.png,CXR995_IM-2478-1002.png,sternotomy sutures and bypass grafts have been...
3298,CXR996_IM-2479-1001.png,CXR996_IM-2479-2001.png,no findings
3299,CXR997_IM-2479-1001.png,CXR997_IM-2479-2001.png,calcified mediastinal no focal areas of consol...
3300,CXR998_IM-2479-1001.png,CXR998_IM-2479-2001.png,cardiomediastinal silhouette demonstrates norm...


In [65]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets
train_data, test_data = train_test_split(df_data, test_size=0.2, random_state=42)

# Splitting the training data into training and validation sets
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Printing the shapes of the resulting datasets
print("Training data shape:", train_data.shape)
print("Validation data shape:", val_data.shape)
print("Testing data shape:", test_data.shape)


Training data shape: (2112, 3)
Validation data shape: (529, 3)
Testing data shape: (661, 3)


In [66]:
DenseNet121_model = DenseNet121(weights='CheXNet_weights.h5', classes = 14, input_shape=(256,256,3))

In [67]:
feature_extraction_model = Model(DenseNet121_model.input, DenseNet121_model.layers[-2].output)

In [68]:
import numpy as np
from PIL import Image
from tensorflow.keras.applications.densenet import preprocess_input
from skimage.transform import resize

def load_image(img_name):
    """
    Loads and preprocesses an image.

    Parameters:
    img_name (str): The path to the image file.

    Returns:
    numpy.ndarray: The preprocessed image array.
    """

    # Load the image
    image = Image.open(img_name)

    # Convert the image to RGB format
    image = image.convert("RGB")

    # Convert the image to numpy array
    X = np.asarray(image)

    # Preprocess the image
    X = preprocess_input(X)

    # Resize the image to (256, 256, 3)
    X = resize(X, (256, 256, 3))

    # Expand the dimensions of the image array
    X = np.expand_dims(X, axis=0)

    return X

In [77]:
def image(data):
    '''
    Convert images into 256 X 256 and generate image features using the CHeXNet model.

    Parameters:
    df (pandas.DataFrame): Input dataframe containing image paths.

    Returns:
    pandas.DataFrame: Output dataframe with added image features.

    Process:
    - Load and preprocess the images.
    - Generate image features using the CHeXNet model.
    - Concatenate the image features.
    - Add the image features to the dataframe.
    '''

    path = 'NLMCXR_png_resized/'
    frontal_xray = data['front X-Ray'].astype(str).tolist()
    lateral_xray = data['lateral X-Ray'].astype(str).tolist()
    

    image_features = []
    for i in tqdm(range(len(frontal_xray))):
        frontal_image = load_image(path + frontal_xray[i])
        lateral_image = load_image(path + lateral_xray[i])
        frontal_image_features = feature_extraction_model.predict(frontal_image)
        lateral_image_features = feature_extraction_model.predict(lateral_image)
        tensor = np.concatenate((frontal_image_features, lateral_image_features), axis=1)
        image_features.append(tensor)
        
    data['image_features'] = image_features
    return data


In [79]:
train_data.head()

,front X-Ray,lateral X-Ray,findings
2383,CXR3606_IM-1781-1001.png,CXR3606_IM-1781-2001.png,cardiac and mediastinal contours are within no...
293,CXR1335_IM-0215-1001.png,CXR1335_IM-0215-1002.png,and lateral chest examination was obtained the...
2808,CXR46_IM-2090-1001.png,CXR46_IM-2090-3003.png,no findings
858,CXR1943_IM-0612-1001.png,CXR1943_IM-0612-2001.png,heart size normal mediastinum unremarkable pul...
2596,CXR3832_IM-1935-1002001.png,CXR3832_IM-1935-1003002.png,the lungs are clear there is no pleural effusi...


In [80]:
new_train_data = image(train_data)

  0%|          | 0/2112 [00:00<?, ?it/s]

1/1 [==============================] - 0s 485ms/step


  0%|          | 1/2112 [00:01<39:11,  1.11s/it]

1/1 [==============================] - 0s 479ms/step


  0%|          | 2/2112 [00:02<38:43,  1.10s/it]

...

100%|█████████▉| 2110/2112 [55:09<00:04,  2.26s/it]

1/1 [==============================] - 1s 1s/step


100%|█████████▉| 2111/2112 [55:12<00:02,  2.26s/it]

1/1 [==============================] - 1s 1s/step


100%|██████████| 2112/2112 [55:14<00:00,  1.57s/it]


In [81]:
new_train_data.head()

,front X-Ray,lateral X-Ray,findings,image_features
2383,CXR3606_IM-1781-1001.png,CXR3606_IM-1781-2001.png,cardiac and mediastinal contours are within no...,"[[0.00026930895, 0.0015878631, 0.0018129623, 0..."
293,CXR1335_IM-0215-1001.png,CXR1335_IM-0215-1002.png,and lateral chest examination was obtained the...,"[[0.00026138927, 0.001149429, 0.0011774194, 0...."
2808,CXR46_IM-2090-1001.png,CXR46_IM-2090-3003.png,no findings,"[[0.00025709017, 0.001940891, 0.0015854018, 0...."
858,CXR1943_IM-0612-1001.png,CXR1943_IM-0612-2001.png,heart size normal mediastinum unremarkable pul...,"[[0.0004861426, 0.0014406334, 0.0022054599, 0...."
2596,CXR3832_IM-1935-1002001.png,CXR3832_IM-1935-1003002.png,the lungs are clear there is no pleural effusi...,"[[0.0005430038, 0.0017081025, 0.0021886774, 0...."


In [82]:
new_test_data = image(test_data)

  0%|          | 0/661 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  0%|          | 1/661 [00:02<25:23,  2.31s/it]

1/1 [==============================] - 1s 1s/step


  0%|          | 2/661 [00:04<25:18,  2.30s/it]

...

100%|█████████▉| 659/661 [27:26<00:05,  2.70s/it]

1/1 [==============================] - 1s 1s/step


100%|█████████▉| 660/661 [27:29<00:02,  2.70s/it]

1/1 [==============================] - 1s 1s/step


100%|██████████| 661/661 [27:32<00:00,  2.50s/it]


In [83]:
new_test_data.head()

,front X-Ray,lateral X-Ray,findings,image_features
52,CXR1052_IM-0040-1001.png,CXR1052_IM-0040-1002.png,the trachea is midline cardio mediastinal silh...,"[[0.0002901905, 0.0018823937, 0.0018676145, 0...."
679,CXR1746_IM-0489-1001.png,CXR1746_IM-0489-1002.png,no findings,"[[0.0003326498, 0.001804881, 0.0019653437, 0.0..."
1253,CXR2381_IM-0941-1001.png,CXR2381_IM-0941-2001.png,cardiac and mediastinal contours are within no...,"[[0.00020333016, 0.0018415797, 0.0025233545, 0..."
1618,CXR2788_IM-1222-1001.png,CXR2788_IM-1222-3001.png,there is no focal consolidation there is no pn...,"[[0.00045017773, 0.0011989961, 0.0015942252, 0..."
203,CXR1225_IM-0150-1001.png,CXR1225_IM-0150-2001.png,heart size is normal there is mild tortuosity ...,"[[0.0003251715, 0.0020210384, 0.0010562495, 0...."


In [84]:
train_data.to_csv('new_train_data.csv', index=False)
test_data.to_csv('new_test_data.csv', index=False)